In [1]:
import pandas as pd
import numpy as np

/var/folders/lj/ms0l79910fz22cg2hchczhw40000gn/T/ipykernel_98710/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 중간생략없애기

In [26]:
pd.set_option('display.max_row', None)
pd.set_option('display.expand_frame_repr', False)

## 전처리 전과 후 describe 비교

In [31]:
feature = pd.read_csv('csv/feature/후보피처(완성본)_6204.csv', index_col=0)
null = pd.read_csv('csv/feature/후보피처(전처리X)_6204.csv', index_col=0)
data = pd.read_csv('csv/재무/차입부채.csv', index_col=0)
null

,Symbol,Name,회계년,label_shifted,결산월,주기,타인자본비율(%),자기자본비율(%),차입금의존도(%),현금및현금성자산구성비율(%),...,ROE(당기순이익)(%),ROE(지배주주순이익)(%),"ROE(지배주주순이익,기말)(%)",자본금영업이익률(%),자본금세전계속사업이익률(%),자본금지배주주순이익률(%),총자산회전율(회),총부채회전율(회),총자본회전율(회),자본금회전율(회)
0,A034730,SK,2020,1,12,Annual,62.33,37.67,39.90,7.33,...,-0.21,1.11,1.09,-499.80,206.33,1230.86,0.60,0.97,1.55,5253.09
1,A900100,애머릿지,2011,0,12,Annual,72.18,27.82,53.77,7.78,...,-45.32,-44.45,-55.72,-16876.10,-32455.06,-32166.25,2.16,3.29,6.31,4559.21
2,A900100,애머릿지,2012,0,12,Annual,72.46,27.54,51.71,10.56,...,-4.67,-4.64,-4.89,224.97,-3939.58,-2619.82,1.97,2.72,7.10,3983.85
3,A900100,애머릿지,2013,0,12,Annual,97.80,2.20,74.50,2.80,...,-192.22,-190.86,-1701.30,-6155.27,-25302.89,-27892.82,1.78,2.13,10.64,1543.50
4,A028260,삼성물산,2016,1,12,Annual,52.53,47.47,16.04,6.07,...,0.11,0.62,0.59,729.28,469.27,561.15,0.65,1.19,1.43,1468.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6199,A115180,큐리언트,2019,0,12,Annual,15.28,84.72,3.79,18.20,...,-79.94,-79.94,-130.61,-477.61,-467.88,-467.88,0.00,0.00,0.00,0.00
6200,A284620,카이노스메드,2018,0,12,Annual,2.07,97.93,0.00,46.08,...,-112.15,-112.15,-99.06,-114.90,-119.57,-119.57,0.00,0.00,0.00,0.00
6201,A284620,카이노스메드,2019,0,12,Annual,11.13,88.87,3.72,68.40,...,-123.85,-123.85,-162.22,-116.13,-115.34,-115.34,0.00,0.00,0.00,0.00
6202,A115180,큐리언트,2020,0,12,Annual,6.33,93.67,1.82,7.63,...,-55.89,-55.44,-36.24,-395.63,-390.90,-385.72,0.00,0.00,0.00,0.00


In [14]:
feature_0 = feature[feature['label_shifted'] == 0].reset_index(drop=True)
feature_1 = feature[feature['label_shifted'] == 1].reset_index(drop=True)
null_0 = null[null['label_shifted'] == 0].reset_index(drop=True)
null_1 = null[null['label_shifted'] == 1].reset_index(drop=True)

In [32]:
feature_columns = feature.columns[6:]

# 숫자가 아닌 값들을 NaN으로 변환
feature[feature_columns] = feature[feature_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
feature[feature_columns] = feature[feature_columns].replace([np.inf, -np.inf], np.nan)

# 'label_shifted'로 그룹화하고 각 그룹별 평균 계산
# 결측치는 평균 계산에서 자동으로 제외됨
feature_means = feature.groupby('label_shifted')[feature_columns].mean().T

# 결과 컬럼명 변경
feature_means.columns = ['feature_0', 'feature_1']
feature_means

,feature_0,feature_1
타인자본비율(%),49.66,48.80
자기자본비율(%),51.15,51.24
차입금의존도(%),29.34,26.61
현금및현금성자산구성비율(%),10.63,9.60
유형자산구성비율(%),28.45,31.90
...,...,...
순운전자본대 총자본비율,0.50,2.09
인건비율,38.74,9.63
매출채권회전율,10.06,7.60
총차입부채회전율증가율(%),42.15,23.28


In [33]:
null_columns = null.columns[6:]

null[null_columns] = null[null_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
null[null_columns] = null[null_columns].replace([np.inf, -np.inf], np.nan)

null_means = null.groupby('label_shifted')[null_columns].mean().T

null_means.columns = ['null_0', 'null_1']
null_means

,null_0,null_1
타인자본비율(%),49.66,48.80
자기자본비율(%),51.90,51.40
차입금의존도(%),29.34,26.61
현금및현금성자산구성비율(%),10.63,9.60
유형자산구성비율(%),28.45,31.90
...,...,...
자본금지배주주순이익률(%),-1831.78,-69.61
총자산회전율(회),0.65,0.81
총부채회전율(회),1.68,2.04
총자본회전율(회),1.83,2.14


In [34]:
feature_means.reset_index(inplace=True)
null_means.reset_index(inplace=True)

In [35]:
feature_means

,index,feature_0,feature_1
0,타인자본비율(%),49.66,48.80
1,자기자본비율(%),51.15,51.24
2,차입금의존도(%),29.34,26.61
3,현금및현금성자산구성비율(%),10.63,9.60
4,유형자산구성비율(%),28.45,31.90
...,...,...,...
85,순운전자본대 총자본비율,0.50,2.09
86,인건비율,38.74,9.63
87,매출채권회전율,10.06,7.60
88,총차입부채회전율증가율(%),42.15,23.28


In [36]:
feature_means['feature_0/1'] = feature_means['feature_0'] / feature_means['feature_1']
null_means['null_0/1'] = null_means['null_0'] / null_means['null_1']

In [37]:
means = pd.merge(feature_means, null_means, on='index', how='left')
means.sort_values(by='feature_0/1')

,index,feature_0,feature_1,feature_0/1,null_0,null_1,null_0/1
22,EBITDA마진율(%),-145.88,2.12,-68.88,-146.41,2.12,-69.06
73,EBITDA(천원)증가율(전년동기)(%),144.47,-4.10,-35.23,NaN,NaN,NaN
76,재무활동으로인한현금흐름(천원)증가율(전년동기)(%),139133.28,-12641.54,-11.01,NaN,NaN,NaN
75,투자활동으로인한현금흐름(천원)증가율(전년동기)(%),-702.93,159.45,-4.41,NaN,NaN,NaN
77,지배주주순이익(천원)증가율(전년동기)(%),817.69,-485.38,-1.68,NaN,NaN,NaN
...,...,...,...,...,...,...,...
20,영업이익률(%),-161.47,-7.14,22.61,-162.06,-7.15,22.67
21,*조정영업이익률(%),-161.51,-7.14,22.63,-162.10,-7.14,22.69
37,자본금지배주주순이익률(%),-1791.38,-69.61,25.74,-1831.78,-69.61,26.32
35,자본금영업이익률(%),-1704.24,-51.23,33.27,-1742.96,-51.23,34.03


In [38]:
means['f0/n0'] = means['feature_0'] / means['null_0']
means['f1/n1'] = means['feature_1'] / means['null_1']
means.sort_values(by='f0/n0')

,index,feature_0,feature_1,feature_0/1,null_0,null_1,null_0/1,f0/n0,f1/n1
55,차입부채/영업이익(배),-11.66,-58.23,0.20,73.43,132.30,0.56,-0.16,-0.44
56,차입부채/EBITDA(배),16.04,6.20,2.59,70.31,35.18,2.00,0.23,0.18
43,유보액/총자산(%),31.72,38.62,0.82,39.86,41.71,0.96,0.80,0.93
53,차입부채조달금리(기말)(%),12.02,8.48,1.42,13.54,11.02,1.23,0.89,0.77
42,유보율(%),5461.21,1214.92,4.50,6147.63,1287.90,4.77,0.89,0.94
...,...,...,...,...,...,...,...,...,...
85,순운전자본대 총자본비율,0.50,2.09,0.24,NaN,NaN,NaN,NaN,NaN
86,인건비율,38.74,9.63,4.02,NaN,NaN,NaN,NaN,NaN
87,매출채권회전율,10.06,7.60,1.32,NaN,NaN,NaN,NaN,NaN
88,총차입부채회전율증가율(%),42.15,23.28,1.81,NaN,NaN,NaN,NaN,NaN


In [32]:
means.to_csv('csv/describe/전처리전후비교(평균).csv', encoding='utf-8-sig')

## 차입부채 관련 컬럼 describe

In [39]:
df = pd.read_csv('csv/재무/차입부채.csv', index_col=0)
list = pd.read_csv('csv/최종기업리스트(6204).csv', index_col=0)

In [40]:
df = pd.merge(list, df, on=['Symbol','Name','회계년'],how='left')
df

,Symbol,Name,회계년,label_shifted,이자비용(천원),영업수익(천원),영업이익(천원),현금흐름(천원),*총차입부채(천원),EBITDA(천원),*총차입부채(천원)_전년도,총차입부채(천원)_평균,차입부채조달금리(평균)(%),차입부채조달금리(기말)(%),차입부채/영업수익(%),차입부채/영업이익(배),차입부채/EBITDA(배),현금흐름/총차입부채(%)
0,A051160,지어소프트,2013,0,87312.00,37186808.00,-5967403.00,-4138050.00,1462220.00,-4932115.00,1000000.00,1231110.00,7.09,5.97,3.93,-0.25,-0.30,-336.12
1,A005980,성지건설,2017,0,639873.00,112427346.00,-4771475.00,-9125541.00,19559244.00,-4653972.00,4162467.00,11860855.50,5.39,3.27,17.40,-4.10,-4.20,-76.94
2,A033640,네패스,2020,0,9351963.00,343559677.00,-3566838.00,23012900.00,451041747.00,68270829.00,223831780.00,337436763.50,2.77,2.07,131.28,-126.45,6.61,6.82
3,A001140,국보,2014,0,1832984.00,97520169.00,-2409123.00,-5417761.00,32739986.00,-881390.00,33119208.00,32929597.00,5.57,5.60,33.57,-13.59,-37.15,-16.45
4,A079190,케스피온,2021,0,471117.00,38949659.00,-4040785.00,-1333328.00,20064966.00,-2337282.00,15174740.00,17619853.00,2.67,2.35,51.52,-4.97,-8.58,-7.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6199,A051160,지어소프트,2016,1,145543.00,64144926.00,-1581945.00,-3957589.00,3563746.00,51100.00,2251755.00,2907750.50,5.01,4.08,5.56,-2.25,69.74,-136.10
6200,A005980,성지건설,2011,1,4116456.00,89899452.00,-5878966.00,59490654.00,13614800.00,-3799231.00,146368490.00,79991645.00,5.15,30.24,15.14,-2.32,-3.58,74.37
6201,A153710,옵티팜,2019,0,158577.00,13652787.00,-1191021.00,201348.00,3557800.00,-334341.00,4410000.00,3983900.00,3.98,4.46,26.06,-2.99,-10.64,5.05
6202,A042940,상지건설,2011,0,1856003.00,31834166.00,-5608798.00,-20043053.00,18762294.00,-5391352.00,21212927.00,19987610.50,9.29,9.89,58.94,-3.35,-3.48,-100.28


In [41]:
df.describe()

/opt/homebrew/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,회계년,label_shifted,이자비용(천원),영업수익(천원),영업이익(천원),현금흐름(천원),*총차입부채(천원),EBITDA(천원),*총차입부채(천원)_전년도,총차입부채(천원)_평균,차입부채조달금리(평균)(%),차입부채조달금리(기말)(%),차입부채/영업수익(%),차입부채/영업이익(배),차입부채/EBITDA(배),현금흐름/총차입부채(%)
count,6204.00,6204.00,6204.00,6204.00,6204.00,6204.00,6204.00,6204.00,6204.00,6204.00,6055.00,6037.00,6200.00,6204.00,6204.00,6204.00
mean,2016.68,0.16,9296664.17,428856599.97,-14557268.86,5019275.38,270108216.56,15979757.78,258992576.54,264550396.55,inf,inf,inf,-18.91,14.51,NaN
std,3.11,0.36,79339906.00,2878255988.93,141926237.80,336239430.84,2528467675.46,370416584.70,2292498435.70,2405650497.15,NaN,NaN,NaN,734.76,1015.32,NaN
min,2011.00,0.00,0.00,0.00,-5846499000.00,-3717432000.00,0.00,-2262643328.00,0.00,0.00,0.00,0.00,0.00,-49454.73,-8766.08,-inf
25%,2014.00,0.00,161372.00,23507722.25,-8409010.75,-10123626.25,8589650.25,-4603406.25,7434501.25,8976949.88,2.76,2.61,19.11,-9.65,-4.69,-53.12
50%,2017.00,0.00,1071064.50,53711988.00,-3353210.50,-2472870.00,24787644.50,-395620.50,23788935.50,24863712.00,4.52,4.49,44.92,-2.70,-0.01,-10.33
75%,2019.00,0.00,2924684.75,146261176.25,-480690.75,2290514.50,64085126.25,3375402.50,61917475.00,62269513.62,7.21,7.54,86.23,-0.02,8.40,7.76
max,2021.00,1.00,2381900000.00,80818755000.00,4086275000.00,13539366000.00,84940016000.00,15634294000.00,74345677000.00,79642846500.00,inf,inf,inf,8364.63,77083.81,inf


In [42]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_columns = df.columns[6:]

# 숫자가 아닌 값들을 NaN으로 변환
df[df_columns] = df[df_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df[df_columns] = df[df_columns].replace([np.inf, -np.inf], np.nan)

# 'label_shifted'로 그룹화하고 각 그룹별 평균 계산
# 결측치는 평균 계산에서 자동으로 제외됨
df_means = df.groupby('label_shifted')[df_columns].mean().T

# 결과 컬럼명 변경
df_means.columns = ['df_0', 'df_1']
df_means

,df_0,df_1
영업이익(천원),-14592955.89,-14363760.92
현금흐름(천원),1292623.14,25226526.32
*총차입부채(천원),237051575.56,449353232.66
EBITDA(천원),11588952.50,39788285.78
*총차입부채(천원)_전년도,227890235.60,427640673.68
총차입부채(천원)_평균,232470905.58,438496953.17
차입부채조달금리(평균)(%),8.66,5.40
차입부채조달금리(기말)(%),12.35,10.50
차입부채/영업수익(%),139.87,65.07
차입부채/영업이익(배),-22.70,1.61


#### test (무시)

In [43]:
fea = feature[['Symbol','Name','회계년','label_shifted','차입부채/영업이익(배)',
'차입부채/EBITDA(배)',
'현금흐름/총차입부채(%)',
'현금흐름/영업수익(%)',
'차입부채조달금리(평균)(%)',
'차입부채조달금리(기말)(%)',
'차입금비율(%)',
'부채비율(%)',
'매출액증가율(전년동기)(%)',
'차입부채/영업수익(%)']]

nu = null[['Symbol','Name','회계년','label_shifted','차입부채/영업이익(배)',
'차입부채/EBITDA(배)',
'현금흐름/총차입부채(%)',
'현금흐름/영업수익(%)',
'차입부채조달금리(평균)(%)',
'차입부채조달금리(기말)(%)',
'차입금비율(%)',
'부채비율(%)',
'매출액증가율(전년동기)(%)',
'차입부채/영업수익(%)']]

da = data[['Symbol','Name','회계년',
'차입부채/영업이익(배)',
'차입부채/EBITDA(배)',
'현금흐름/총차입부채(%)',
'차입부채조달금리(평균)(%)',
'차입부채조달금리(기말)(%)',
# '현금흐름/영업수익(%)',
# '차입금비율(%)',
# '부채비율(%)',
# '매출액증가율(전년동기)(%)',
'차입부채/영업수익(%)']]

In [21]:
aa = pd.merge(fea,nu,on=['Symbol','Name','회계년','label_shifted'], how='left')
aa.to_csv('aa.csv',encoding='utf-8-sig')

In [44]:
bb = pd.merge(aa,da,on=['Symbol','Name','회계년'], how='left')
bb.to_csv('bb.csv', encoding='utf-8-sig')